In [1]:
import operator
import re

from pyspark import SparkConf, SparkContext

In [2]:
sc = SparkContext(conf=SparkConf().setAppName("FindPath").setMaster("local"))

In [5]:
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

def step(item):
    prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
    return (next_v, prev_d + 1)

def complete(item):
    v, old_d, new_d = item[0], item[1][0], item[1][1]
    return (v, old_d if old_d is not None else new_d)

n = 400  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()


In [6]:
x = 12
d = 0
distances = sc.parallelize([(x, d)]).partitionBy(n)

while True:
    candidates = distances.join(forward_edges, n).map(step)
    new_distances = distances.fullOuterJoin(candidates, n).map(complete, True).persist()
    count = new_distances.filter(lambda i: i[1] == d + 1).count()
    if count > 0:
        d += 1
        distances = new_distances
        print("d = ", d, "count = ", count)
    else:
        break

('d = ', 1, 'count = ', 4)
('d = ', 2, 'count = ', 1)
('d = ', 3, 'count = ', 7)
('d = ', 4, 'count = ', 2)
('d = ', 5, 'count = ', 3)
('d = ', 6, 'count = ', 3)
('d = ', 7, 'count = ', 4)
('d = ', 8, 'count = ', 2)
